In [13]:
import numpy as np
import pandas as pd
import string
import math
import random
from sklearn import svm

In [3]:
df = pd.read_csv("/Users/Ellie/Documents/Kaggle/Titanic/train.csv")
result = pd.read_csv("/Users/Ellie/Documents/Kaggle/Titanic/gender_submission.csv")
testing = pd.read_csv("/Users/Ellie/Documents/Kaggle/Titanic/test.csv")
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Getting substring for title
def getSubstring(string, substrings):
    for s in substrings:
        if string.find(s) != -1:
            return s
    return np.nan
       

# Replacing all titles with mr, mrs, miss, master
def replace_titles(x):
    title = x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
# Replacing missing value for age 
def replace_ages(x, ageGroup):
    age = x['Age']
    if math.isnan(age):
        return ageGroup[x['Pclass']][x['SibSp']]
    else:
        return age


In [5]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer'] 
df['Title'] = df['Name'].map(lambda x: getSubstring(x, title_list))
df['Title'] = df.apply(replace_titles, axis = 1)

In [6]:
# Fill the missing values
ageGroup = df.groupby(['Pclass', 'SibSp'])['Age'].mean()
ageGroup[3][8] = df['Age'][df['Pclass'] == 3].mean()
df['Age'] = df.apply(lambda x: replace_ages(x, ageGroup), axis = 1)

In [7]:
# Replace carbin number
cabin_null = df['Cabin'].isnull()
cabin_list = [i for i in range(0, len(df['Cabin'])) if cabin_null[i] == True]
df['Cabin'][cabin_list] = 'Unknown'
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
df['Cabin']=df['Cabin'].map(lambda x: getSubstring(x, cabin_list))

/Users/Ellie/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
# Embark only 2 missing values
df = df[df['Embarked'].isnull() == False]
df = df.reset_index(drop=True)

In [46]:
df['Sex_male'] = (df['Sex'] == 'male').astype('int')
df['Title_Master'] = (df['Title'] == 'Master').astype('int')
df['Title_Miss'] = (df['Title'] == 'Miss').astype('int')
df['Title_Mr'] = (df['Title'] == 'Mr').astype('int')
df['Title_Mrs'] = (df['Title'] == 'Mrs').astype('int')
df['Cabin_A'] = (df['Cabin'] == 'A').astype('int')
df['Cabin_B'] = (df['Cabin'] == 'B').astype('int')
df['Cabin_C'] = (df['Cabin'] == 'C').astype('int')
df['Cabin_D'] = (df['Cabin'] == 'D').astype('int')
df['Cabin_E'] = (df['Cabin'] == 'E').astype('int')
df['Cabin_F'] = (df['Cabin'] == 'F').astype('int')
df['Cabin_G'] = (df['Cabin'] == 'G').astype('int')
df['Cabin_T'] = (df['Cabin'] == 'T').astype('int')
df['Cabin_Unknown'] = (df['Cabin'] == 'Unknown').astype('int')
df['Embarked_C'] = (df['Embarked'] == 'C').astype('int')
df['Embarked_Q'] = (df['Embarked'] == 'Q').astype('int')
df['Embarked_S'] = (df['Embarked'] == 'S').astype('int')


In [56]:
train.head(3)

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_Unknown,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
2,3,26.0,0,0,7.9250,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [58]:
data = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked', 'Sex', 'Survived', 'Title'], axis = 1)
response = df['Survived']

In [31]:
# perform cross validation, 5 folds
cv_id = []
for i in range(0, 5):
    cv_id.extend([i] * 142)
cv_id.extend([0] * 1) 
random.shuffle(cv_id)

In [32]:
for i in range(0,5):
    train_index = [index for index in range(0, len(df)) if index != i]
    test_index = [index for index in range(0, len(df)) if index == i]
    train = data.iloc[train_index]
    train_response = response.iloc[train_index]
    test = data.iloc[test_index]
    test_response = response.iloc[test_index]
    

    

711

In [21]:
int(len(df) * 0.8)

711